In [1]:
from yahooquery import Ticker
import pandas as pd
import bs4 as bs
import pickle
import requests
import time

def save_russell1000_tickers():
    resp = requests.get('https://en.wikipedia.org/wiki/Russell_1000_Index')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []

    for row in table.findAll('tr')[1:]:        
        ticker = row.findAll('td')[1].text.rstrip()
        tickers.append(ticker)
        
    with open("russell1000tickers.pickle","wb") as f:
        pickle.dump(tickers,f)
        
    return tickers

In [15]:
tickers=save_russell1000_tickers()
tickers.sort()
# tickers

In [26]:
retry=[]

In [27]:
dfResult=pd.DataFrame()
counter=0
FINAL_COUNT=1000

flag = True
last=''
for t in tickers:    
    if not flag and t != last:
        #print('skipping ticker ', t)        
        continue
    elif not flag and t == last:
        flag = True
        continue
    else:
        time.sleep(2)  
        print('processing ticker',t)
        if counter==FINAL_COUNT:
            break
        counter=counter+1
    
    t=t.replace('.','-')
    ticker=Ticker(t)
    balanceSheet=ticker.balance_sheet(frequency="q")
    
    try:
        balanceSheet=balanceSheet[balanceSheet['periodType']=="3M"].sort_values('asOfDate').groupby('symbol').tail(1)
        # print(balanceSheet)
    except: 
        print('balanceSheet error:',balanceSheet)
        retry.append(t)
        time.sleep(30)
        continue
        
    incomeStatement=ticker.income_statement(frequency="q")
    try:
        incomeStatement=incomeStatement[incomeStatement['periodType']=="TTM"].sort_values('asOfDate').groupby('symbol').tail(1)
        # print(incomeStatement)
    except:
        print('incomeStatement error:',incomeStatement)
        retry.append(t)
        time.sleep(30)
        continue

    try:
        valuationMeasure = pd.DataFrame.from_dict(ticker.summary_detail).loc[['marketCap']].transpose()
        valuationMeasure.index.name = 'symbol'
        valuationMeasure.columns=['MarketCap']
        # print(valuationMeasure)
    except:
        print('valuationMeasure error:', t)
        retry.append(t)
        continue
    
    mergedBsIs = pd.merge(balanceSheet, incomeStatement, on=['asOfDate','symbol']).fillna(0)
    if mergedBsIs.empty:        
        print('bad join 1:',t)
        retry.append(t)
        
    mergedBsIsSum = pd.merge(mergedBsIs, valuationMeasure, on=['symbol']).fillna(0)
    if mergedBsIsSum.empty:
        print('bad join 2:',t)
        retry.append(t)
    
    if 'EBIT' not in mergedBsIsSum.columns:
        print('missing EBIT:', t)
        continue
        
    if 'InvestedCapital' not in mergedBsIsSum.columns:
        print('missing InvestedCapital:', t)
        continue
    
    roic = mergedBsIsSum.EBIT / mergedBsIsSum.InvestedCapital
    try:
        cash = mergedBsIsSum.CashAndCashEquivalents
    except:
        print('cash error: ',t)
        continue
    cdl = mergedBsIsSum.CurrentDeferredLiabilities if 'CurrentDeferredLiabilities' in mergedBsIsSum.columns else 0
    ltdclb = mergedBsIsSum.LongTermDebtAndCapitalLeaseObligation if 'LongTermDebtAndCapitalLeaseObligation' in mergedBsIsSum.columns else 0
    ncdl = mergedBsIsSum.NonCurrentDeferredLiabilities if 'NonCurrentDeferredLiabilities' in mergedBsIsSum.columns else 0
    oncl = mergedBsIsSum.OtherNonCurrentLiabilities if 'OtherNonCurrentLiabilities' in mergedBsIsSum.columns else 0
    debt = cdl+ltdclb+ncdl+oncl
    mergedBsIsSum['totalDebt']=debt
    
    try:
        preferredequity = mergedBsIsSum.CapitalStock-mergedBsIsSum.CommonStock
        mergedBsIsSum['preferredequity']=preferredequity
    except:
        print('capital or common stock missing:', t)
        continue
    
    try:
        networth = mergedBsIsSum.InvestedCapital+cash-debt-preferredequity
    except:
        print('InvestedCapital missing:',t)
        continue
        
    faustmannRatio = mergedBsIsSum.MarketCap/networth
    
    dfRoic = roic.to_frame()
    dfFaustmannRatio = faustmannRatio.to_frame()
    dfRoicToFaustmann = (roic/faustmannRatio).to_frame()
    
    dfRoic.columns=['roic']
    dfFaustmannRatio.columns=['faustmannRatio']
    dfRoicToFaustmann.columns=['roicToFaustmann']

    result = pd.merge(
    pd.merge(
        pd.merge(mergedBsIsSum,dfRoic,on=['symbol']),
        dfFaustmannRatio,
        on=['symbol']
    ),
    dfRoicToFaustmann,
    on=['symbol']
    )
    test=result[['asOfDate','EBIT','InvestedCapital','roic','MarketCap','CashAndCashEquivalents','totalDebt','preferredequity','faustmannRatio']]    
    dfResult=pd.concat([dfResult,test])
dfResult

processing ticker A
processing ticker AA
processing ticker AAL
processing ticker AAP
processing ticker AAPL
processing ticker ABBV
processing ticker ABC
processing ticker ABMD
processing ticker ABT
processing ticker ACC
processing ticker ACGL
processing ticker ACHC
processing ticker ACM
processing ticker ACN
processing ticker ADBE
processing ticker ADI
processing ticker ADM
processing ticker ADNT
processing ticker ADP
processing ticker ADS
processing ticker ADSK
processing ticker ADT
processing ticker AEE
processing ticker AEP
processing ticker AES
processing ticker AFG
processing ticker AFL
processing ticker AGCO
processing ticker AGIO
processing ticker AGNC
missing EBIT: AGNC
processing ticker AGO
processing ticker AGR
processing ticker AIG
processing ticker AIV
processing ticker AIZ
processing ticker AJG
processing ticker AKAM
processing ticker AL
processing ticker ALB
processing ticker ALGN
processing ticker ALK
processing ticker ALKS
processing ticker ALL
processing ticker ALLE
pr

,asOfDate,EBIT,InvestedCapital,roic,MarketCap,CashAndCashEquivalents,totalDebt,preferredequity,faustmannRatio
symbol,,,,,,,,,
A,2021-10-31,1.441000e+09,8.118000e+09,0.177507,47303495680,1.484000e+09,3.829000e+09,0.0,8.193919
AA,2021-09-30,1.517000e+09,5.603000e+09,0.270748,9130627072,1.452000e+09,2.248000e+09,0.0,1.899444
AAL,2021-09-30,-2.458000e+09,3.116000e+10,-0.078883,11661726720,2.930000e+08,5.968200e+10,0.0,-0.413112
AAP,2021-09-30,8.840850e+08,4.230537e+09,0.208977,14967741440,6.046450e+08,3.712407e+09,0.0,13.331025
AAPL,2021-09-30,1.118520e+11,1.878090e+11,0.595563,2944128516096,3.494000e+10,1.453540e+11,0.0,38.040294
...,...,...,...,...,...,...,...,...,...
ZBRA,2021-09-30,9.750000e+08,3.807000e+09,0.256107,32842473472,3.070000e+08,1.831000e+09,0.0,14.385665
ZEN,2021-09-30,-1.649720e+08,1.574325e+09,-0.104789,12206985216,5.325170e+08,1.487686e+09,0.0,19.715524
ZG,2021-09-30,-5.622600e+07,1.021156e+10,-0.005506,15270435840,2.195366e+09,2.010585e+09,0.0,1.468828


In [28]:
dfResult.to_csv('austrian.csv', sep=';')

In [29]:
retry

['ALXN',
 'ARD',
 'BPYU',
 'CHK',
 'CHK',
 'CLGX',
 'CLNY',
 'CMD',
 'CY',
 'DATA',
 'EGN',
 'EV',
 'EVHC',
 'FDC',
 'FLIR',
 'GDI',
 'GRA',
 'HRB',
 'HRB',
 'JEC',
 'LB',
 'LM',
 'LPT',
 'MSG',
 'MXIM',
 'MYL',
 'PF',
 'PRAH',
 'RBC',
 'RP',
 'SERV',
 'STAY',
 'STI',
 'TCF',
 'TCO',
 'TPCO',
 'TSRO',
 'ULTI',
 'USG',
 'VAR',
 'VSM',
 'VVC',
 'WBC',
 'WCG',
 'WFTLF',
 'WP',
 'WYND']

In [30]:
dfResult=pd.DataFrame()
for t in retry:    
    if not flag and t != last:
        #print('skipping ticker ', t)        
        continue
    elif not flag and t == last:
        flag = True
        continue
    else:
        # print('processing ticker ',t)        

        if counter==FINAL_COUNT:
            break
        counter=counter+1
    
    t=t.replace('.','-')
    ticker=Ticker(t)
    balanceSheet=ticker.balance_sheet(frequency="q")
    
    try:
        balanceSheet=balanceSheet[balanceSheet['periodType']=="3M"].sort_values('asOfDate').groupby('symbol').tail(1)
    except: 
        print('balanceSheet error:',balanceSheet)
        continue
        
    incomeStatement=ticker.income_statement(frequency="q")
    try:
        incomeStatement=incomeStatement[incomeStatement['periodType']=="TTM"].sort_values('asOfDate').groupby('symbol').tail(1)
    except:
        print('incomeStatement error:',incomeStatement)
        continue

    try:
        valuationMeasure = pd.DataFrame.from_dict(ticker.summary_detail).loc[['marketCap']].transpose()
        valuationMeasure.index.name = 'symbol'
        valuationMeasure.columns=['MarketCap']
    except:
        print('valuationMeasure error:', t)
        continue
    
    mergedBsIs = pd.merge(balanceSheet, incomeStatement, on=['asOfDate','symbol','periodType']).fillna(0)
    mergedBsIsSum = pd.merge(mergedBsIs, valuationMeasure, on=['symbol']).fillna(0)
    
    if 'EBIT' not in mergedBsIsSum.columns:
        print('missing EBIT:', t)
        continue
        
    if 'InvestedCapital' not in mergedBsIsSum.columns:
        print('missing InvestedCapital:', t)
        continue
    
    roic = mergedBsIsSum.EBIT / mergedBsIsSum.InvestedCapital
    try:
        cash = mergedBsIsSum.CashAndCashEquivalents
    except:
        print('cash error: ',t)
        continue
    cdl = mergedBsIsSum.CurrentDeferredLiabilities if 'CurrentDeferredLiabilities' in mergedBsIsSum.columns else 0
    ltdclb = mergedBsIsSum.LongTermDebtAndCapitalLeaseObligation if 'LongTermDebtAndCapitalLeaseObligation' in mergedBsIsSum.columns else 0
    ncdl = mergedBsIsSum.NonCurrentDeferredLiabilities if 'NonCurrentDeferredLiabilities' in mergedBsIsSum.columns else 0
    oncl = mergedBsIsSum.OtherNonCurrentLiabilities if 'OtherNonCurrentLiabilities' in mergedBsIsSum.columns else 0
    debt = cdl+ltdclb+ncdl+oncl
    mergedBsIsSum['totalDebt']=debt
    
    try:
        preferredequity = mergedBsIsSum.CapitalStock-mergedBsIsSum.CommonStock
        mergedBsIsSum['preferredequity']=preferredequity
    except:
        print('capital or common stock missing:', t)
        continue
    
    try:
        networth = mergedBsIsSum.InvestedCapital+cash-debt-preferredequity
    except:
        print('InvestedCapital missing:',t)
        continue
        
    faustmannRatio = mergedBsIsSum.MarketCap/networth
    
    dfRoic = roic.to_frame()
    dfFaustmannRatio = faustmannRatio.to_frame()
    dfRoicToFaustmann = (roic/faustmannRatio).to_frame()
    
    dfRoic.columns=['roic']
    dfFaustmannRatio.columns=['faustmannRatio']
    dfRoicToFaustmann.columns=['roicToFaustmann']

    result = pd.merge(
    pd.merge(
        pd.merge(mergedBsIsSum,dfRoic,on=['symbol']),
        dfFaustmannRatio,
        on=['symbol']
    ),
    dfRoicToFaustmann,
    on=['symbol']
    )
    test=result[['asOfDate','EBIT','InvestedCapital','roic','MarketCap','CashAndCashEquivalents','totalDebt','preferredequity','faustmannRatio']]    
    dfResult=pd.concat([dfResult,test])
dfResult

balanceSheet error: Balance Sheet data unavailable for ALXN
balanceSheet error: Balance Sheet data unavailable for ARD
balanceSheet error: Balance Sheet data unavailable for BPYU
balanceSheet error: Balance Sheet data unavailable for CLGX
balanceSheet error: Balance Sheet data unavailable for CLNY
balanceSheet error: Balance Sheet data unavailable for CMD
balanceSheet error: Balance Sheet data unavailable for CY
balanceSheet error: Balance Sheet data unavailable for DATA
balanceSheet error: Balance Sheet data unavailable for EGN
balanceSheet error: Balance Sheet data unavailable for EV
balanceSheet error: Balance Sheet data unavailable for EVHC
balanceSheet error: Balance Sheet data unavailable for FDC
balanceSheet error: Balance Sheet data unavailable for FLIR
balanceSheet error: Balance Sheet data unavailable for GDI
balanceSheet error: Balance Sheet data unavailable for GRA
balanceSheet error: Balance Sheet data unavailable for JEC
balanceSheet error: Balance Sheet data unavailable 

,asOfDate,EBIT,InvestedCapital,roic,MarketCap,CashAndCashEquivalents,totalDebt,preferredequity,faustmannRatio
symbol,,,,,,,,,


In [57]:
dfResult.to_csv('austrian_retry.csv', sep=';')